Faster R-CNN

Import  (tutorial: https://github.com/tensorflow/models/blob/master/object_detection/object_detection_tutorial.ipynb)

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

from densenet import densenet_model


Using TensorFlow backend.


In [2]:
# This is needed to display the images.
%matplotlib inline
sys.path.append(os.path.abspath('./tf_models'))
sys.path.append(os.path.abspath('./tf_models/slim'))

In [3]:
#READ finetuned FASTER_RCNN model
# Path to frozen detection graph. This is the actual model that is used for the object detection.
obj_detection_graph_path = 'models/faster_rcnn/output/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
labels = "data/label_map.pbtxt"
num_classes = 2

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(obj_detection_graph_path, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
    
label_map = label_map_util.load_labelmap(labels)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [4]:
#READ finetuned DENSENET MODEL
densenet_img_rows, densenet_img_cols = 224, 224  # Resolution of inputs
channel = 3
classification_model_path = 'models/densenet/densenet_finetuned.h5'
classification_model = densenet_model(img_rows=densenet_img_rows, img_cols=densenet_img_cols,
                                        color_type=channel, num_classes=num_classes, weights_path=classification_model_path)


/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:182: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), use_bias=False, strides=(2, 2), name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:330: UserWarning: The `merge` function is deprecated and will be removed after

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv3_8_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv3_9_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv3_9_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bia

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv4_11_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv4_11_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv4_12_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), 

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv4_26_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv4_27_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv4_27_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv5_5_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv5_5_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv5_6_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv5_20_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv5_20_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv5_21_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), 

COMPILED


In [5]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

#transform_img_for_densenet
def detected_obj_box_to_densenet_input_images(image, boxes, densenet_img_rows, densenet_img_cols):
    #gets full image, and boxes that faster rcnn labeled as objects
    #returns list of individual images  (densenet_img_rows x densenet_img_cols)
    
    #box [y_min, x_min, y_max, x_max]  normalized values
       
    im_width = image.shape[1]
    im_height = image.shape[0]
    images = list()
    for box in boxes[0][:10]:
        #denormalize
        x_min = int(box[1]*im_width)
        x_max = int(box[3]*im_width)
        y_min = int(box[0]*im_height)
        y_max = int(box[2]*im_height)
        obj = image[y_min:y_max, x_min:x_max] #img[y: y + h, x: x + w]
        img = cv2.resize(obj, (densenet_img_rows, densenet_img_cols)).astype(np.float32)
        print (img[0][0][0])
        img[:, :, 0] -= 103.939
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 123.68
        images.append(img)

    return np.array(images)

def interpred_predictions(predictions,boxes):
    #gets densenet prediction and object box coordinates
    #returns only vehicle box coordinates, confidence score and class list
    score_list=list()
    class_list = list()
    box_list = list()
    i=0
    for pred in predictions:
        if pred[0]<pred[1]:
            print("pred: %s"%i)
            score_list.append(pred[1])
            class_list.append(1)
            box_list.append(boxes[i])
        i+=1
    return (np.array(box_list),np.array(score_list),np.array(class_list))



In [6]:
#input video
cap = cv2.VideoCapture('data/videos/test_video.mp4')
#output video
video = cv2.VideoWriter('data/video.avi',-1,1,(1280,720))

#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#video = cv2.VideoWriter("output12.avi", fourcc, 20.0, (1280,720))
    
i = 0
while(cap.isOpened()):
    print('\n\nframe %s'%i)
    i +=1
    if i >37:
        break
    #elif i >3:
    #    break
    ret, image = cap.read()
    image_np  = np.asarray(image)
    
    #FASTER R_CNN OBJECT DETECTION
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]           
                image_np_expanded = np.expand_dims(image_np, axis=0)

                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Each box represents a part of the image where a particular object was detected.
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Each score represent how level of confidence for each of the objects.
                # Score is shown on the result image, together with the class label.
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                
                # Actual detection.
                (boxes, scores, classes, num_detections) = sess.run(
                  [boxes, scores, classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
            
    #DENSENET  OBJECT CLASSIFICATION    
    images=detected_obj_box_to_densenet_input_images(image_np, boxes, densenet_img_rows, densenet_img_cols)
    predictions = classification_model.predict(images, batch_size=1, verbose=1)
    (vehicle_boxes, scores, classes) = interpred_predictions(predictions,boxes[0])   
    vis_util.visualize_boxes_and_labels_on_image_array(image_np,vehicle_boxes,classes,scores, category_index,use_normalized_coordinates=True,line_thickness=2)
    video.write(image_np)

cv2.destroyAllWindows()
video.release()



frame 0
45.0
50.0
48.0
30.0
107.0
66.0
132.0
78.0
68.0
20.0
10/10 [==============================] - 21s    
pred: 0
pred: 1
pred: 2
pred: 6
pred: 7
pred: 8


frame 1
49.0
40.0
24.0
26.0
81.0
59.0
54.0
24.0
84.0
42.0
10/10 [==============================] - 23s    
pred: 0
pred: 1
pred: 5
pred: 8
pred: 9


frame 2
41.0
26.0
23.0
19.0
45.0
89.0
53.0
51.0
16.0
16.0
10/10 [==============================] - 19s    
pred: 0
pred: 4
pred: 5
pred: 6


frame 3
42.0
24.0
25.0
44.0
78.0
81.0
69.0
92.0
98.0
49.0
10/10 [==============================] - 21s    
pred: 0
pred: 1
pred: 2
pred: 3
pred: 4
pred: 5
pred: 6
pred: 7


frame 4
35.0
35.0
28.0
22.0
49.0
148.0
78.0
94.0
94.0
50.0
10/10 [==============================] - 20s    
pred: 0
pred: 1
pred: 4
pred: 5


frame 5
41.0
43.0
36.0
83.0
44.0
96.0
90.0
78.0
72.0
50.0
10/10 [==============================] - 21s    
pred: 0
pred: 4
pred: 6
pred: 7
pred: 8


frame 6
31.0
34.0
37.0
18.0
55.0
90.0
83.0
101.0
81.0
80.0
10/10 [===================